In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit, train_test_split

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

# RF Clasificador Casas

In [175]:
properati = properati.loc[properati['property_type'] == 'house']
properati.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2751 entries, 4 to 48322
Data columns (total 26 columns):
created_on                    2751 non-null object
property_type                 2751 non-null object
place_name                    2751 non-null object
state_name                    2751 non-null object
lat-lon                       2751 non-null object
lat                           2751 non-null float64
lon                           2751 non-null float64
price                         2751 non-null float64
currency                      2751 non-null object
price_aprox_local_currency    2751 non-null float64
price_aprox_usd               2751 non-null float64
surface_total_in_m2           2751 non-null float64
surface_covered_in_m2         2751 non-null float64
price_usd_per_m2              2751 non-null float64
price_per_m2                  2751 non-null float64
floor                         1 non-null float64
rooms                         1134 non-null float64
expenses         

In [176]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Pruebo con np.histogram

In [177]:
cant_buckets = 50

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_per_m2'],rango)

#properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

[     0.     1976.18   3952.36   5928.54   7904.72   9880.9   11857.08
  13833.26  15809.44  17785.62  19761.8   21737.98  23714.16  25690.34
  27666.52  29642.7   31618.88  33595.06  35571.24  37547.42  39523.6
  41499.78  43475.96  45452.14  47428.32  49404.5   51380.68  53356.86
  55333.04  57309.22  59285.4   61261.58  63237.76  65213.94  67190.12
  69166.3   71142.48  73118.66  75094.84  77071.02  79047.2   81023.38
  82999.56  84975.74  86951.92  88928.1   90904.28  92880.46  94856.64
  96832.82  98809.  ]


In [178]:
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.loc[:,('created_on')] = pd.to_datetime(properati['created_on'])
properati['anio'] = [x.year for x in properati['created_on'] ]

properati.groupby('categories_by_price').count().sort_values('created_on',ascending=False)

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range,anio
categories_by_price,,,,,,,,,,,,,,,,,,,,,
11.0,213,213,213,213,213,213,213,213,213,213,...,213,213,213,213,213,213,213,213,213,213
9.0,201,201,201,201,201,201,201,201,201,201,...,201,201,201,201,201,201,201,201,201,201
10.0,183,183,183,183,183,183,183,183,183,183,...,183,183,183,183,183,183,183,183,183,183
14.0,179,179,179,179,179,179,179,179,179,179,...,179,179,179,179,179,179,179,179,179,179
8.0,176,176,176,176,176,176,176,176,176,176,...,176,176,176,176,176,176,176,176,176,176
12.0,168,168,168,168,168,168,168,168,168,168,...,168,168,168,168,168,168,168,168,168,168
13.0,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161
7.0,136,136,136,136,136,136,136,136,136,136,...,136,136,136,136,136,136,136,136,136,136
6.0,133,133,133,133,133,133,133,133,133,133,...,133,133,133,133,133,133,133,133,133,133


In [179]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['categories_by_price']

### Busco hiper-parametros con random search

In [171]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

oob_score=[True,False]
criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=20,n_iter=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 118.34 segundos para 5 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.375 (std: 0.045)
Promedio fit time: 1.578s
Hiper-parametros: {'n_estimators': 60, 'max_features': None, 'oob_score': True, 'criterion': 'entropy'}

Puesto: 2
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.370 (std: 0.040)
Promedio fit time: 0.818s
Hiper-parametros: {'n_estimators': 70, 'max_features': 'auto', 'oob_score': False, 'criterion': 'entropy'}

Puesto: 3
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.369 (std: 0.043)
Promedio fit time: 0.591s
Hiper-parametros: {'n_estimators': 50, 'max_features': 'log2', 'oob_score': False, 'criterion': 'entropy'}

Puesto: 4
Promedio training score: 0.992 (std: 0.001)
Promedio validation score: 0.361 (std: 0.038)
Promedio fit time: 0.961s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'log2', 'oob_score': False, 'criterion': 'entro

<IPython.core.display.Javascript object>

In [172]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.05679378  0.0469424   0.11615543  0.06550298  0.09035526  0.26879457
  0.15110547  0.12768706  0.07666306]


In [173]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.35208711433756806

### RF Regresor

In [206]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['price_aprox_local_currency']

In [211]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse','mae']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=20,n_iter=20) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 491.05 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.966 (std: 0.003)
Promedio validation score: 0.805 (std: 0.092)
Promedio fit time: 0.253s
Hiper-parametros: {'n_estimators': 60, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.964 (std: 0.004)
Promedio validation score: 0.799 (std: 0.110)
Promedio fit time: 1.188s
Hiper-parametros: {'n_estimators': 80, 'max_features': 'log2', 'criterion': 'mae'}

Puesto: 3
Promedio training score: 0.965 (std: 0.004)
Promedio validation score: 0.795 (std: 0.099)
Promedio fit time: 0.336s
Hiper-parametros: {'n_estimators': 40, 'max_features': None, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.965 (std: 0.005)
Promedio validation score: 0.795 (std: 0.106)
Promedio fit time: 0.269s
Hiper-parametros: {'n_estimators': 50, 'max_features': 'log2', 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.964 (std: 0.003)
Promedio validation score: 0.794 (

<IPython.core.display.Javascript object>

In [212]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.06140288  0.05644572  0.09462327  0.04465251  0.05877776  0.17336684
  0.31409236  0.12227021  0.07436844]


In [215]:
print mejor_rf.predict(X_test)-y_test

22280      9200.000000
15014    149000.000000
34455     25750.000000
43097       450.000000
15608    -45016.666667
37473   -433333.333333
48304     12476.666667
23916    -15383.333333
30693    -17516.666667
23557      7100.000000
14028     73250.000000
7055      15233.333333
6427      11616.666667
17841     -1500.000000
8199     101283.333333
12264    143133.333333
39857    119881.666667
690      137265.000000
22339    117960.734127
41183     65300.000000
10338    -59091.666667
40339    -22166.666667
11619    -25916.666667
40457     23379.800000
17275     48998.333333
29928      7666.666667
40984     20183.333333
15882    103400.000000
10349      3863.333333
22181    157483.333333
             ...      
12501    138233.333333
34549      7800.000000
14142     53358.333333
39693    -39183.333333
16914     25613.333333
42251    -32722.222222
39623     -3851.791727
20661   -486160.000000
21818   -526833.333333
18796     59627.777778
27073     19258.416667
4781       1103.888889
9717      1